## Movie Recommender 
All the helper files are already loaded here. No need to import!
Data files are in the current folder.

In [7]:
"""
movie_data_helper.py
Original author: Felix Sun (6.008 TA, Fall 2015)
Modified by:
- Danielle Pace (6.008 TA, Fall 2016),
- George H. Chen (6.008/6.008.1x instructor, Fall 2016)

***Do not modify this file.***

This file has a number of helper files for the movie rating problem, to
interact with the data files and return relevant information.
"""

import numpy as np
from os.path import isfile
from sys import exit


def get_movie_id_list():
    """
    This function returns a 1D NumPy array of all movie ID's based on data in
    './data/'.

    Output
    ------
    1D NumPy array of all the movie ID's
    """

    return np.loadtxt("data/movieNames.dat",
                      dtype='int32',
                      delimiter='\t',
                      usecols=(0,))


def get_movie_name(movie_id):
    """
    Gets a movie name given a movie ID.

    Input
    -----
    - movie_id: integer from 0 to <number of movies - 1> specifying which movie

    Output
    ------
    - movie_name: string containing the movie name
    """

    # -------------------------------------------------------------------------
    # ERROR CHECK
    #
    filename_to_check = "data/ratingsMovie%d.dat" % movie_id
    if not(isfile(filename_to_check)):
        exit('Movie ID %d does not exist' % movie_id)
    #
    # END OF ERROR CHECK
    # -------------------------------------------------------------------------

    filename = "data/movieNames.dat"
    movies = np.loadtxt(filename,
                        dtype={'names': ('movieid', 'moviename'),
                               'formats': ('int32', 'S100')},
                        delimiter='\t')
    movie_indices = [i for i in range(len(movies))
                     if (movies[i]['movieid'] == movie_id)]
    movie_name = movies[movie_indices]['moviename'][0].lstrip()
    return movie_name


def get_ratings(movie_id):
    """
    Gets all the ratings for a given movie.

    Input
    -----
    - movie_id: integer from 0 to <number of movies - 1> specifying which movie

    Output
    ------
    - ratings: 1D array consisting of the ratings for the given movie
    """

    filename = "data/ratingsMovie%d.dat" % movie_id

    # -------------------------------------------------------------------------
    # ERROR CHECK
    #
    if not(isfile(filename)):
        exit('Movie ID %d does not exist' % movie_id)
    #
    # END OF ERROR CHECK
    # -------------------------------------------------------------------------

    data = np.loadtxt(filename,
                      dtype={'names': ('userid', 'rating'),
                             'formats': ('int32', 'int32')},
                      delimiter='\t')
    ratings = data['rating']
    return ratings


Code to compute Posterior Probability

In [8]:

import matplotlib.pyplot as plt

import numpy as np
import scipy
import scipy.misc
from sys import exit


def compute_posterior(prior, likelihood, y):
    """
    Use Bayes' rule for random variables to compute the posterior distribution
    of a hidden variable X, given N observations Y_0, Y_1, ..., Y_{N-1}.
    Conditioned on X, these observations Y_0, Y_1, ..., Y_{N-1} are i.i.d.

    Hidden random variable X is assumed to take on a value in {0, 1, ..., M-1}.

    Each random variable Y_i takes on a value in {0, 1, ..., K-1}.

    Inputs
    ------
    - prior: a length M vector stored as a 1D NumPy array; prior[m] gives the
        (unconditional) probability that X = m
    - likelihood: a K row by M column matrix stored as a 2D NumPy array;
        likelihood[k, m] gives the probability that Y = k given X = m
    - y: a length-N vector stored as a 1D NumPy array; y[n] gives the observed
        value for random variable Y_n

    Output
    ------
    - posterior: a length M vector stored as a 1D NumPy array: posterior[m]
        gives the probability that X = m given
        Y_0 = y_0, ..., Y_{N-1} = y_{n-1}
    """

    # -------------------------------------------------------------------------
    # ERROR CHECKS -- DO NOT MODIFY
    #

    # check that prior probabilities sum to 1
    if np.abs(1 - np.sum(prior)) > 1e-06:
        exit('In compute_posterior: The prior probabilities need to sum to 1')

    # check that likelihood is specified as a 2D array
    if len(likelihood.shape) != 2:
        exit('In compute_posterior: The likelihood needs to be specified as ' +
             'a 2D array')

    K, M = likelihood.shape

    # make sure likelihood and prior agree on number of hidden states
    if len(prior) != M:
        exit('In compute_posterior: Mismatch in number of hidden states ' +
             'according to the prior and the likelihood.')

    # make sure the conditional distribution given each hidden state value sums
    # to 1
    for m in range(M):
        if np.abs(1 - np.sum(likelihood[:, m])) > 1e-06:
            exit('In compute_posterior: P(Y | X = %d) does not sum to 1' % m)

    #
    # END OF ERROR CHECKS
    # -------------------------------------------------------------------------

    # -------------------------------------------------------------------------
    # YOUR CODE GOES HERE FOR PART (b)
    #
    # Place your code to compute the log of the posterior here: store it in a
    # NumPy array called `log_answer`. If you exponentiate really small
    # numbers, the result is likely to underflow (i.e., it will be so small
    # that the computer will just make it 0 rather than storing the right
    # value). You need to go to log-domain. Hint: this next line is a good
    # first step.
    log_prior = np.log(prior)
    log_likelihood = np.log(likelihood)
    total_log_likelihood = np.zeros((1,M))
    for obser in range(len(y)):
        total_log_likelihood = total_log_likelihood + log_likelihood[y[obser],:]
    
    log_answer = (log_prior + total_log_likelihood) - scipy.misc.logsumexp(log_prior + total_log_likelihood)
    
    
    #
    # END OF YOUR CODE FOR PART (b)
    # -------------------------------------------------------------------------

    
    
    # do not exponentiate before this step
    posterior = np.exp(log_answer)
    return posterior


In [9]:
def compute_movie_rating_likelihood(M):
    """
    Compute the rating likelihood probability distribution of Y given X where
    Y is an individual rating (takes on a value in {0, 1, ..., M-1}), and X
    is the hidden true/inherent rating of a movie (also takes on a value in
    {0, 1, ..., M-1}).

    Please refer to the instructions of the project to see what the
    likelihood for ratings should be.

    Output
    ------
    - likelihood: an M row by M column matrix stored as a 2D NumPy array;
        likelihood[k, m] gives the probability that Y = k given X = m
    """

    # define the size to begin with
    likelihood = np.zeros((M, M))

    # -------------------------------------------------------------------------
    # YOUR CODE GOES HERE FOR PART (c)
    #
    # Remember to normalize the likelihood, so that each column is a
    # probability distribution.
    #

    #
    # END OF YOUR CODE FOR PART (c)
    # -------------------------------------------------------------------------
    for yi in range(M):
        for ax in range(M):
            if yi == ax:
                likelihood[ax,yi] = 2.0
            else:
                likelihood[ax, yi] = 1.0/np.abs(yi - ax)
        likelihood[:, yi] /= np.sum(likelihood[:, yi])
    
    return likelihood
(compute_movie_rating_likelihood(3))

array([[ 0.57142857,  0.25      ,  0.14285714],
       [ 0.28571429,  0.5       ,  0.28571429],
       [ 0.14285714,  0.25      ,  0.57142857]])

In [10]:
def infer_true_movie_ratings(num_observations=-1):
    """
    For every movie, computes the posterior distribution and MAP estimate of
    the movie's true/inherent rating given the movie's observed ratings.

    Input
    -----
    - num_observations: integer that specifies how many available ratings to
        use per movie (the default value of -1 indicates that all available
        ratings will be used)

    Output
    ------
    - posteriors: a 2D array consisting of the posterior distributions where
        the number of rows is the number of movies, and the number of columns
        is M, i.e., the number of possible ratings (remember ratings are
        0, 1, ..., M-1); posteriors[i] gives a length M vector that is the
        posterior distribution of the true/inherent rating of the i-th movie
        given ratings for the i-th movie (where for each movie, the number of
        observations used is precisely what is specified by the input variable
        `num_observations`)
    - MAP_ratings: a 1D array with length given by the number of movies;
        MAP_ratings[i] gives the true/inherent rating with the highest
        posterior probability in the distribution `posteriors[i]`
    """

    M = 11  # all of our ratings are between 0 and 10
    prior = np.array([1.0 / M] * M)  # uniform distribution
    likelihood = compute_movie_rating_likelihood(M)

    # get the list of all movie IDs to process
    movie_id_list = get_movie_id_list()
    num_movies = len(movie_id_list)

    # -------------------------------------------------------------------------
    # YOUR CODE GOES HERE FOR PART (d)
    #
    # Your code should iterate through the movies. For each movie, your code
    # should:
    #   1. Get all the observed ratings for the movie
    #   2. Use the ratings you retrieved and the function compute_posterior to
    #      obtain the posterior of the true/inherent rating of the movie
    #      given the observed ratings
    #   3. Find the rating for each movie that maximizes the posterior

    # These are the output variables - it's your job to fill them.
    posteriors = np.zeros((num_movies, M))
    MAP_ratings = np.zeros(num_movies)

    for movie_id in movie_id_list:
        y = get_ratings(movie_id)
        posteriors[movie_id,:] = compute_posterior(prior, likelihood,y)
        MAP_ratings[movie_id]= compute_posterior(prior, likelihood,y).argmax()
        
    #
    # END OF YOUR CODE FOR PART (d)
    # -------------------------------------------------------------------------

    return posteriors, MAP_ratings

a,b = infer_true_movie_ratings(num_observations=-1)
c = b.max()
l = []
for i in range(len(b)):
    if b[i] == c:
        l.append(i)
        
for li in l:
    print(get_movie_name(li))

b'Sense and Sensibility (1995)'
b'Usual Suspects, The (1995)'
b'Postman, The (Postino, Il) (1994)'
b'Braveheart (1995)'
b'Taxi Driver (1976)'
b'Hoop Dreams (1994)'
b'Star Wars: Episode IV - A New Hope (a.k.a. Star Wars) (1977)'
b'Pulp Fiction (1994)'
b'Shawshank Redemption, The (1994)'
b'Forrest Gump (1994)'
b"Schindler's List (1993)"
b'Blade Runner (1982)'
b'Silence of the Lambs, The (1991)'
b'Fargo (1996)'
b'Wallace & Gromit: A Close Shave (1995)'
b'Dr. Strangelove or: How I Learned to Stop Worrying and Love '
b'Godfather, The (1972)'
b"Singin' in the Rain (1952)"
b'Vertigo (1958)'
b'Rear Window (1954)'
b'North by Northwest (1959)'
b'Some Like It Hot (1959)'
b'Casablanca (1942)'
b'Maltese Falcon, The (1941)'
b'Wizard of Oz, The (1939)'
b'Gone with the Wind (1939)'
b'Citizen Kane (1941)'
b'2001: A Space Odyssey (1968)'
b"It's a Wonderful Life (1946)"
b'African Queen, The (1951)'
b'Sound of Music, The (1965)'
b'Monty Python and the Holy Grail (1975)'
b'Wallace & Gromit: The Wrong Trous